In [2]:
import re
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
from typing import Dict, List, get_type_hints
from IPython.display import display
# ! pip install spacy
# python -m spacy download en_core_web_sm

In [3]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [4]:
df = pd.read_csv('生成所有matched句子/Dofloo_report_match_sent.csv')
print('shape',df.shape)
df.head()

shape (34, 4)


,report,baseline,sentence number,sentence
0,Dofloo-MalwareMustDie,/etc,38,1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a7...
1,Dofloo-MalwareMustDie,/etc/.,38,1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a7...
2,Dofloo-MalwareMustDie,/etc,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."
3,Dofloo-MalwareMustDie,/etc/.,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."
4,Dofloo-MalwareMustDie,nul,40,".text:0x0a840 mov r1, #0x42\n.text:0x0a844 ..."


In [5]:
df.loc[0, 'sentence']

"1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n.text:0x0a760  stmfd   sp!, {r4-r8,lr}\n.text:0x0a764  sub  sp, sp, #0x208\n.text:0x0a768  add  r7, sp, #0x108\n.text:0x0a76c  mov  r4, r0\n.text:0x0a770  mov  r8, r1\n.text:0x0a774  mov  r0, r7\n.text:0x0a778  mov  r1, #0x100\n.text:0x0a77c  bl   sub_0x026fb0\n.text:0x0a780  ldr  r0,  <-- chattr -i /etc/crontab\n.text:0x0a784  bl   sub_0x0e3e0\n.text:0x0a788  ldr  r0, <-- chmod +w /etc/crontab\n\nadding the autostart entry in it:\n\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n.text:0x0a7dc ; xref: sub_0x0a760\n.text:0x0a7dc  ldr  r1, <-- sed -i '/%s/d' /etc/crontab\n.text:0x0a7e0  mov  r2, r5\n.text:0x0a7e4  mov  r0, r6\n   :\n.text:0x0a800  ldr  r1, <-- echo '*/1 * * * * root %s/%s %s' >> /etc/crontab\n.text:0x0a804  str  r8, [sp,#0x108+var_108]\n.text:0x0a808  bl   sub_0x0182dc\n.text:0x0a80c  mov  r0, r6\n.text:0x0a810  bl   sub_0x0e3e0\n\ncreate the file: /etc/.mysys, which later on found as the self copy attempt."

In [4]:
def select_target_sentence(ST: str, paragraph: str) -> str:
    '''若 input text 有多行，回傳第一行含有 ST 的句子'''
    if paragraph.find('\n') == -1:
        return paragraph
    for sent in paragraph.split('\n'):
        if sent.find(ST) != -1:
            return sent
    return None

In [5]:
# https://stackoverflow.com/questions/66636097/prevent-spacy-tokenizer-from-splitting-on-specific-character
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, HYPHENS
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

nlp = spacy.load("en_core_web_sm")
text = "Get 10ct/liter off when using our App"
# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        #r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer
doc = nlp(text)
print([t.text for t in doc])

['Get', '10ct/liter', 'off', 'when', 'using', 'our', 'App']


In [28]:
lenx = len(df)
ST_list = []
Path("./images").mkdir(parents=True, exist_ok=True)
for no in range(lenx):
    ST = df.loc[no, 'baseline']
    target_text = select_target_sentence(ST, df.loc[no, 'sentence'])
    sentence_nlp = nlp(target_text)
    svg = displacy.render(sentence_nlp, style="dep", jupyter=False)
    output_path = Path(f"./images/no{no}_dependency_plot.svg") # 檔名不能用 Spacial token 因為含有 /
    ST_list.append(f'{no:<2} {ST:<25} {target_text}')
    # output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w", encoding="utf-8") as f:
        f.write(svg)
with open('./images/ST_list.txt', 'w', encoding="utf-8") as f:
    for id,metadata in enumerate(ST_list):
        f.write(f'{metadata}\n')

In [51]:
no = 6
ST = df.loc[no, 'baseline']
target_text = select_target_sentence(ST, df.loc[no, 'sentence'])
print('Special token:',ST,'|',target_text)

Special token: /etc | trend micro researchers also discovered that the latest variant of the aesddos bot can modify files i.e., /etc/rc.local and /etc/rc.d/rc.local, as an autostart technique by appending the {malware path}/{malware file name} reboot command.


In [43]:
doc = nlp(target_text)
for token in doc:
    # print(token.text, token.tag_, token.head.text, token.dep_)
    print(f'{token.text: <12} {token.tag_:<4} {token.head.text:<10} {token.dep_}')

trend        NNP  micro      compound
micro        NNP  researchers compound
researchers  NNS  discovered nsubj
also         RB   discovered advmod
discovered   VBD  discovered ROOT
that         IN   modify     mark
the          DT   variant    det
latest       JJS  variant    amod
variant      NN   modify     nsubj
of           IN   variant    prep
the          DT   aesddos    det
aesddos      NNS  of         pobj
bot          NN   variant    appos
can          MD   modify     aux
modify       VB   discovered ccomp
files        NNS  modify     dobj
i.e.         FW   modify     advmod
,            ,    modify     punct
/etc/rc.local JJ   modify     conj
and          CC   modify     cc
/etc/rc.d/rc.local RB   modify     conj
,            ,    /etc/rc.d/rc.local punct
as           IN   modify     prep
an           DT   technique  det
autostart    NN   technique  compound
technique    NN   as         pobj
by           IN   technique  prep
appending    VBG  by         pcomp
the          DT

In [59]:
token_id = -1
for i,token in enumerate(doc):
    if token.text.find(ST) >= 0:
        token_id = i
print(token_id, doc[token_id])

20 /etc/rc.d/rc.local


In [61]:
print([token.text for token in doc[token_id].rights])
list(doc[token_id].subtree)

[',']


[/etc/rc.d/rc.local, ,]

In [44]:
# displacy.serve(doc, style='dep')
displacy.render(doc, style='dep', jupyter=True)

In [20]:
# 斷句
about_doc = nlp(target_text)
sentences = list(about_doc.sents)
len(sentences)

for sentence in sentences:
    print('#',sentence)

# .text:0x0a780  ldr  r0,  <-- chattr -i /etc/crontab


In [7]:
target_text = "the virtual file /proc/cpuinfo is opened and read in words"
doc = nlp(target_text)
for token in doc:
    # print(token.text, token.tag_, token.head.text, token.dep_)
    print(f'{token.text: <14} {token.tag_:<4} {token.head.text:<10} {token.dep_}')

the            DT   file       det
virtual        JJ   file       amod
file           NN   opened     nsubjpass
/proc/cpuinfo  :    file       punct
is             VBZ  opened     auxpass
opened         VBN  opened     ROOT
and            CC   opened     cc
read           VBN  opened     conj
in             IN   read       prep
words          NNS  in         pobj
